# 05. Going Modular
Follow the class and code

In [2]:
from helper_functions import plot_predictions, plot_decision_boundary, accuracy_fn
import mlxtend
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
import os
import pandas as pd
from pathlib import Path
import random
import requests
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import torch
from torch import nn
from torchmetrics import Accuracy, ConfusionMatrix
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import Tuple, Dict, List
import zipfile



c:\Users\Tiger\anaconda3\envs\p311Udacity1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Make dicrectory

In [3]:
os.makedirs('going_modular', exist_ok=True)

In [4]:
# set up paths
data_path = Path('../data/')
image_path = data_path /'pizza_steak_sushi_20_percent'
train_dir = image_path/'train'
test_dir = image_path/'test'
train_dir, test_dir

(WindowsPath('../data/pizza_steak_sushi_20_percent/train'),
 WindowsPath('../data/pizza_steak_sushi_20_percent/test'))

In [53]:
%%writefile going_modular/data_loaders.py
'''
function for creating dataloader
'''
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

def create_dataloaders(
            train_dir: str,
            test_dir: str,
            transform: transforms.Compose,
            batch_size: int,
            num_workers: int= 0,
            pin_memory = True
            ):
    
    """_summary_

    Returns:
        _type_: _description_
    """    """_summary_
    """    

    train_data = datasets.ImageFolder(train_dir, transform = transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    class_names = train_data.classes

    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle = True,
        num_workers = num_workers,
        pin_memory = pin_memory
    )

    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle = False,
        num_workers = num_workers,
        pin_memory=pin_memory
    )

    return train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_loaders.py


## create model

In [19]:
%%writefile going_modular/hassan_TinyVGG.py
# # do the CNN to classify into one of e classes
#  with model stacking to improve performance
import torch
from torch import nn
class HassanFood(nn.Module):
    """_summary_

    Args:
        nn (_type_): _description_
    """    
    def __init__ (self, input_shape, hidden_units, output_shape,
                  in_ConvNN_ker_size: int = 3,
                  in_ConvNN_stirde:int = 1,
                  in_ConvNN_pad: int = 1,
                  in_MAXP_KerSize: int = 4,
                  in_MAXP_stride = 1,
                  in_batch_size = 32):
        super().__init__()
        self.conv1= nn.Conv2d(
                in_channels = input_shape,
                out_channels = hidden_units,
                kernel_size = in_ConvNN_ker_size,
                stride = in_ConvNN_stirde,
                padding = in_ConvNN_pad 
                )
        self.conv2 = nn.Conv2d(
            in_channels = hidden_units,
            out_channels=hidden_units,
            kernel_size=in_ConvNN_ker_size,
            stride=in_ConvNN_stirde,
            padding=in_ConvNN_pad
        )
        self.relu = nn.LeakyReLU()
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(kernel_size=in_MAXP_KerSize,
                                    stride = in_MAXP_stride)
        self.lazydense = nn.LazyLinear(out_features=output_shape)

    def forward (self, x):
        x = self.lazydense(self.flatten(self.maxpool(self.relu(self.conv2(self.maxpool(self.relu(self.conv1(x))))))))

        return (x)
    


Overwriting going_modular/hassan_TinyVGG.py


In [ ]:
# testing it. 
torch.manual_seed(42)
from going_modular import hassan_TinyVGG
device = 'cuda' if torch.cuda.is_available() else 'cpu'
out_shape = 3
model_custom_1 = hassan_TinyVGG.HassanFood(input_shape = 3,
                            hidden_units = 8,
                            output_shape = out_shape).to(device)
model_custom_1

## Creating train and test step

In [79]:
%%writefile going_modular/engine.py 
'''
training and test steps
'''
from typing import Dict, List, Tuple
import torch
from tqdm.auto import tqdm


def train_step (model: torch.nn.Module,
                dataloader: torch.utils.data.DataLoader,
                loss_fn: torch.nn.Module,
                optimizer: torch.optim.Optimizer,
                device:torch.device):

    """_summary_

    Returns:
        _type_: _description_
    """    
    model.train()
    train_loss, train_acc = 0,  0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn (y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim = 1), dim= 1)
        train_acc  += (y_pred_class == y).sum().item()/len(y_pred)
        
    train_loss = train_loss/len(dataloader)
    train_acc = train_acc /len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):

    """_summary_

    Returns:
        _type_: _description_
    """    
    model.eval()
    
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)
            loss = loss_fn (test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
    test_loss = test_loss/ len(dataloader)
    test_acc = test_acc / len (dataloader)
    return test_loss, test_acc


def train_model(model: torch.nn.Module,
                train_dataloader: torch.utils.data.DataLoader,
                test_dataloader:torch.utils.data.DataLoader,
                optimizer: torch.optim.Optimizer,
                device: torch.device,
                loss_fn: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                epochs: int=1
                ):

    """_summary_

    Returns:
        _type_: _description_
    """    
    results = {'train_loss':[],
               'train_acc': [],
               'test_loss':[],
               'test_acc':[]}
    
    range_epochs = range(epochs)
    for epoch in tqdm(range_epochs):
        train_loss, train_acc = train_step(model=model,
                                           dataloader = train_dataloader,
                                           loss_fn = loss_fn,
                                           optimizer=optimizer,
                                           device = device)
        test_loss, test_acc = test_step (model=model,
                                        dataloader=test_dataloader,
                                        loss_fn = loss_fn,
                                        device = device)
        
        if epoch % 10 == 0:
            print (
                f'Epoch: {epoch+1} |'
                f'train_loss: {train_loss:.4f} |'
                f'train_acc:{train_acc:.4f} | '
                f'test_loss: {test_loss:.4f} | '
                f'test_acc: {test_acc:.4f}'
            )
            
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)
        
    return results

Overwriting going_modular/engine.py


## Utils


In [85]:
%%writefile going_modular/utils.py
'''
utils form odel
'''
from pathlib import Path
import torch

def save_model (model: torch.nn.Module,
            target_dir: str,
            model_name: str):
    """_summary_

    Args:
        model (torch.nn.Module): _description_
        target_dir (str): _description_
        model_name (str): _description_
    """
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok = True)

    assert model_name.endswith('.pth') or model_name.endswith('pt'), 'model_name shoule end with ".pt", or ".pth"'
    model_save_path = target_dir_path/model_name

    print (f'[INFO] Saving model to: {model_save_path}')
    torch.save(obj = model.state_dict(),
                f=model_save_path)


Overwriting going_modular/utils.py


## test make sure model runs


In [ ]:
# import os
# import sys
# import torch
# from torchvision import transforms
# import importlib

# from going_modular import data_loaders, engine, hassan_TinyVGG, utils
# sys.path.append('./going_modular') 
# importlib.reload(data_loaders)
# importlib.reload(engine)
# importlib.reload(hassan_TinyVGG)
# importlib.reload(utils)
# # from going_modular importlib.reload(ibkr_helper)
# # importlib.reload(general_helper)

# torch.manual_seed(42)
# torch.cuda.manual_seed(42)


# NUM_EPOCS = 5
# BATCH_SIZE = 32
# HIDDEN_UNITS = 10
# LEARING_RATE = 0.001

# train_dir = '../data/pizza_steak_sushi_20_percent/train'
# test_dir = '../data/pizza_steak_sushi_20_percent/test'

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# data_transform = transforms.Compose([
#     transforms.Resize(size = (224, 224)),
#                       transforms.ToTensor()])

# train_dataloader, test_datalaoder, class_names = data_loaders.create_dataloaders(
#     train_dir=train_dir,
#     test_dir = test_dir,
#     transform=data_transform,
#     batch_size=BATCH_SIZE

# )

# hassan_custom_model  = hassan_TinyVGG.HassanFood(
#                         input_shape = 3,
#                         hidden_units = 9,
#                         output_shape= len(class_names)).to(device)
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params=hassan_custom_model.parameters(), lr = 0.001)

# from timeit import default_timer as timer 
# start_time = timer ()

# model_0_results = engine.train_model(model=hassan_custom_model,
#                                      train_dataloader=train_dataloader,
#                                      test_dataloader=test_datalaoder,
#                                      loss_fn = loss_fn,
#                                      optimizer=optimizer,
#                                      epochs=NUM_EPOCS,
#                                      device=device)

# end_time = timer()
# print(f'[INFO] total train time : {end_time - start_time:.3f} seconds')

# utils.save_model (model = hassan_custom_model,
#            target_dir = 'models',
#             model_name = '05_hassan_model_0.pth' )

In [95]:
%%writefile going_modular/train.py
import os
import sys
import torch
from torchvision import transforms
import importlib

# from going_modular import data_loaders, engine, hassan_TinyVGG, utils
import data_loaders, engine, hassan_TinyVGG, utils
# sys.path.append('./going_modular') 
# importlib.reload(data_loaders)
# importlib.reload(engine)
# importlib.reload(hassan_TinyVGG)
# importlib.reload(utils)
# from going_modular importlib.reload(ibkr_helper)
# importlib.reload(general_helper)

torch.manual_seed(42)
torch.cuda.manual_seed(42)


NUM_EPOCS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARING_RATE = 0.001

train_dir = '../data/pizza_steak_sushi_20_percent/train'
test_dir = '../data/pizza_steak_sushi_20_percent/test'

# train_dir = '../../data/pizza_steak_sushi_20_percent/train'
# test_dir = '../../data/pizza_steak_sushi_20_percent/test'

print(os.path.abspath(train_dir))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data_transform = transforms.Compose([
    transforms.Resize(size = (224, 224)),
                      transforms.ToTensor()])

train_dataloader, test_datalaoder, class_names = data_loaders.create_dataloaders(
    train_dir=train_dir,
    test_dir = test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE

)

hassan_custom_model  = hassan_TinyVGG.HassanFood(
                        input_shape = 3,
                        hidden_units = 9,
                        output_shape= len(class_names)).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=hassan_custom_model.parameters(), lr = 0.001)

from timeit import default_timer as timer 
start_time = timer ()

model_0_results = engine.train_model(model=hassan_custom_model,
                                     train_dataloader=train_dataloader,
                                     test_dataloader=test_datalaoder,
                                     loss_fn = loss_fn,
                                     optimizer=optimizer,
                                     epochs=NUM_EPOCS,
                                     device=device)

end_time = timer()
print(f'[INFO] total train time : {end_time - start_time:.3f} seconds')

utils.save_model (model = hassan_custom_model,
           target_dir = 'models',
            model_name = '05_hassan_model_0.pth' )

Overwriting going_modular/train.py


In [96]:
!python going_modular/train.py

e:\Dropbox\GithubRepo\Udemy\pytorch-deep-learning-main\pytorch-deep-learning-main - Copy\data\pizza_steak_sushi_20_percent\train
Epoch: 1 |train_loss: 5.2459 |train_acc:0.3187 | test_loss: 1.3941 | test_acc: 0.4716
[INFO] total train time : 15.893 seconds
[INFO] Saving model to: models\05_hassan_model_0.pth


c:\Users\Tiger\anaconda3\envs\p311Udacity1\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '

100%|██████████| 5/5 [00:15<00:00,  3.18s/it]


## Use argparse

In [105]:
%%writefile going_modular/train_argparse.py
import argparse
import os
import sys
import torch
from torchvision import transforms
import importlib

# from going_modular import data_loaders, engine, hassan_TinyVGG, utils
import data_loaders, engine, hassan_TinyVGG, utils
# sys.path.append('./going_modular') 
# importlib.reload(data_loaders)
# importlib.reload(engine)
# importlib.reload(hassan_TinyVGG)
# importlib.reload(utils)
# from going_modular importlib.reload(ibkr_helper)
# importlib.reload(general_helper)

parser = argparse.ArgumentParser(description = 'Get Hyper peramters')

parser.add_argument('--num_epochs',
                    default = 5,
                    type= int,
                    help = 'number of training epochs, default = 5')

parser.add_argument ('--batch_size',
                    default = 16,
                    type=int,
                    help='number of hidden units in layers, default = 16')

parser.add_argument ('--hidden_units',
                    default = 8,
                    type = int,
                    help = 'number of hidden units, default = 16')

parser.add_argument ('--learning_rate',
                    default = 0.001,
                    type = float,
                    help = 'learning rate, default = 0.001')

parser.add_argument ('--train_dir',
                    default = '../data/pizza_steak_sushi_20_percent/train',
                    type = str,
                    help = 'training directory default = ../data/pizza_steak_sushi_20_percent/train' )

parser.add_argument ('--test_dir',
                    default = '../data/pizza_steak_sushi_20_percent/test',
                    type = str,
                    help = 'test directory default = ../data/pizza_steak_sushi_20_percent/test ')

args = parser.parse_args()


torch.manual_seed(42)
torch.cuda.manual_seed(42)


NUM_EPOCS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARING_RATE = args.learning_rate

train_dir = args.train_dir
test_dir = args.test_dir

# train_dir = '../../data/pizza_steak_sushi_20_percent/train'
# test_dir = '../../data/pizza_steak_sushi_20_percent/test'

print(os.path.abspath(train_dir))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data_transform = transforms.Compose([
    transforms.Resize(size = (224, 224)),
                      transforms.ToTensor()])

train_dataloader, test_datalaoder, class_names = data_loaders.create_dataloaders(
    train_dir=train_dir,
    test_dir = test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE

)

hassan_custom_model  = hassan_TinyVGG.HassanFood(
                        input_shape = 3,
                        hidden_units = 9,
                        output_shape= len(class_names)).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=hassan_custom_model.parameters(), lr = 0.001)

from timeit import default_timer as timer 
start_time = timer ()

model_0_results = engine.train_model(model=hassan_custom_model,
                                     train_dataloader=train_dataloader,
                                     test_dataloader=test_datalaoder,
                                     loss_fn = loss_fn,
                                     optimizer=optimizer,
                                     epochs=NUM_EPOCS,
                                     device=device)

end_time = timer()
print(f'[INFO] total train time : {end_time - start_time:.3f} seconds')

utils.save_model (model = hassan_custom_model,
           target_dir = 'models',
            model_name = '05_hassan_model_0.pth' )

Overwriting going_modular/train_argparse.py


In [109]:
!python going_modular/train_argparse.py --batch_size 16

e:\Dropbox\GithubRepo\Udemy\pytorch-deep-learning-main\pytorch-deep-learning-main - Copy\data\pizza_steak_sushi_20_percent\train
Epoch: 1 |train_loss: 3.8187 |train_acc:0.4483 | test_loss: 1.1703 | test_acc: 0.4938
[INFO] total train time : 18.205 seconds
[INFO] Saving model to: models\05_hassan_model_0.pth


c:\Users\Tiger\anaconda3\envs\p311Udacity1\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '

100%|██████████| 5/5 [00:18<00:00,  3.64s/it]


## 